In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import os
import random
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split


In [14]:
window_size = 60  # Number of frames; adjust based on your data
slide_step = 8  # Adjust for finer or coarser granularity

In [39]:

class RdmDataset(Dataset):
    def __init__(self, root_dir, event_labels_df, window_size, train=True):
        self.data = []
        self.labels = []
        self.window_size = window_size
        # Assuming 'event_labels_df' is a DataFrame with columns ['RADAR_capture', 'Start_Frame', 'frame_stable', 'frame_break', 'End_Frame']
        self.event_labels_df = event_labels_df

        folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        print(folders)
        split_idx = int(len(folders) * 0.8)  # 80-20 split for train-test
        selected_folders = folders[:split_idx] if train else folders[split_idx:]
        
        for folder in selected_folders:
            folder_path = os.path.join(root_dir, folder)
            for file in os.listdir(folder_path):
                if file.endswith('.npy'):
                    rdm_data = np.load(os.path.join(folder_path, file))
                    print(rdm_data.shape)
                    if rdm_data.shape[1] != 23 or rdm_data.shape[2] != 13:
                        continue
                    print(file)
                    capture_labels = self.event_labels_df[self.event_labels_df['RADAR_capture'] == file[:14]]
                    for index, row in capture_labels.iterrows():
                        # Create windows and labels based on 'Start_Frame' to 'frame_stable' for GOUP, and 'frame_break' to 'End_Frame' for DOWN
                        self._create_windows_and_labels(rdm_data, row)

    def _create_windows_and_labels(self, rdm_data, row):
        # Generate windows for GOUP
        for start in range(int(row['Start_Frame']), int(row['frame_stable']) - self.window_size + 1):
            self.data.append(rdm_data[start:start+self.window_size, :, :])
            self.labels.append(0)  # GOUP label

        # Generate windows for DOWN
        # Check if 'frame_break' is NaN before generating windows for DOWN
        if not np.isnan(row['frame_break']):
            # Generate windows for DOWN
            for start in range(int(row['frame_break']), int(row['End_Frame']) - self.window_size + 1):
                self.data.append(rdm_data[start:start+self.window_size, :, :])
                self.labels.append(2)  # DOWN label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
# Define the model
class RdmClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(RdmClassifier, self).__init__()
        # Define a simple CNN architecture
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # Assuming RDMs have a single channel
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),  # This will flatten the output of the convolutional layers
        )
        cnn_output_size = self._get_conv_output_size()

        # Define the LSTM layer
        self.lstm = nn.LSTM(cnn_output_size, hidden_size, batch_first=True)
        
        # Define the fully connected layer for classification
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        batch_size, seq_len, _, _ = x.size()
        # Apply CNN to each RDM in the sequence
        c_out = self.cnn(x.view(batch_size * seq_len, 1, *x.size()[-2:]))
        
        # Reshape for LSTM input
        r_out = c_out.view(batch_size, seq_len, -1)
        
        # Pack the sequence for LSTM
        packed_input = pack_padded_sequence(r_out, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed_input)

        # Use the last hidden state
        out = self.fc(hidden[-1])
        return out
    
    def _get_conv_output_size(self):
        # We can create a dummy input to calculate the output size after the CNN layers
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 23, 13)  # Replace with your RDM shape
            dummy_output = self.cnn(dummy_input)
            return dummy_output.size(-1)



In [41]:
import torch.optim as optim

def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for data, labels in train_loader:
            data = data.float()  # Convert data to float
            optimizer.zero_grad()
            outputs = model(data, torch.tensor([data.size(1)]*len(data), dtype=torch.long))  # Ensure lengths tensor matches type
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Example setup
model = RdmClassifier(num_classes=3, hidden_size=128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


event_labels_df = pd.read_csv('/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES.csv')

full_dataset_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch"
train_dataset = RdmDataset(full_dataset_path, event_labels_df, window_size=window_size)

# Assuming 'train_dataset' is an instance of 'RdmDataset'
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

print('Number of samples in train_dataset:', len(train_dataset))

# Train the model
train_model(model, train_loader, criterion, optimizer)


['01', '02', '03', '04', '05', '08', '10', '12', '13', '14', '15', '16', '18', '22', '24']
(1000, 23, 13)
01_MNTRL_RR_V1_channel1.npy
(1000, 23, 13)
01_MNTRL_RR_V1_channel2.npy
(1000, 23, 13)
01_MNTRL_RR_V1_channel3.npy
(1000, 23, 13)
01_MNTRL_RR_V1_channel4.npy
(1000, 23, 13)
01_MNTRL_RR_V2_channel1.npy
(1000, 23, 13)
01_MNTRL_RR_V2_channel2.npy
(1000, 23, 13)
01_MNTRL_RR_V2_channel3.npy
(1000, 23, 13)
01_MNTRL_RR_V2_channel4.npy
(1000, 23, 13)
01_MNTRL_RR_V3_channel1.npy
(1000, 23, 13)
01_MNTRL_RR_V3_channel2.npy
(1000, 23, 13)
01_MNTRL_RR_V3_channel3.npy
(1000, 23, 13)
01_MNTRL_RR_V3_channel4.npy
(1000, 23, 13)
01_MNTRR_RR_V1_channel1.npy
(1000, 23, 13)
01_MNTRR_RR_V1_channel2.npy
(1000, 23, 13)
01_MNTRR_RR_V1_channel3.npy
(1000, 23, 13)
01_MNTRR_RR_V1_channel4.npy
(1000, 23, 13)
01_MNTRR_RR_V2_channel1.npy
(1000, 23, 13)
01_MNTRR_RR_V2_channel2.npy
(1000, 23, 13)
01_MNTRR_RR_V2_channel3.npy
(1000, 23, 13)
01_MNTRR_RR_V2_channel4.npy
(1000, 23, 13)
01_MNTRR_RR_V3_channel1.npy
(1000,

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [35]:
def slide_and_predict(model, data, window_size, slide_step):
    predictions = []
    for start in range(0, len(data) - window_size + 1, slide_step):
        end = start + window_size
        window_data =  data[start:end]
        prediction = model.predict(window_data)  # Adjust based on your model's prediction method
        predictions.append((start, end, prediction))
    return predictions

In [ ]:
from torch.utils.data import Dataset, DataLoader

class RadarDataset(Dataset):
    def __init__(self, segments, labels):
        self.segments = segments  # Your preprocessed and segmented RADAR data
        self.labels = labels  # Corresponding labels for each segment

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        return self.segments[idx], self.labels[idx]

# Example usage
dataset = RadarDataset(segments, labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [34]:
def interpret_predictions(predictions, event_labels):
    events_detected = []
    for start, end, prediction in predictions:
        if prediction == 'GOUP' and any((label['Start_Frame'] <= start < label['frame_stable']) or
                                        (label['Start_Frame'] < end <= label['frame_stable']) for _, label in event_labels.iterrows()):
            events_detected.append((start, end, 'GOUP'))
        elif prediction == 'DOWN' and any((label['frame_break'] <= start < label['End_Frame']) or
                                          (label['frame_break'] < end <= label['End_Frame']) for _, label in event_labels.iterrows()):
            events_detected.append((start, end, 'DOWN'))
    return events_detected


In [37]:
# Assuming processed_data is loaded for each RADAR capture
# Example for a single RADAR capture; loop or functionize for all captures as needed
radar_capture_name = '01_MNTRL_RR_V1_Channel1'
participant = radar_capture_name[:2]
processed_data = np.load(f'/Volumes/FourTBLaCie/Yoga_Study_RADAR_1Ch/{participant}/{radar_capture_name}.npy')  # Adjust path as necessary
event_labels = event_labels_df[event_labels_df['RADAR_capture'] == radar_capture_name[:14]]

# Parameters for sliding window
window_size = 100  # Example size, adjust as needed
slide_step = 10  # Example step size, adjust as needed

# Predict events
predictions = slide_and_predict(model, processed_data, window_size, slide_step)
events_detected = interpret_predictions(predictions, event_labels)

# Do something with detected events
for event in events_detected:
    print(event)

AttributeError: 'RdmClassifier' object has no attribute 'predict'